### Do stuff with curated bouts

In [1]:
import os
import glob
import socket
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal

### Fuck matplotlib, I'm using poltly now
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info('Running on {}'.format(socket.gethostname()))

2021-07-05 23:27:12,701 root         INFO     Running on pakhi


In [15]:
from ceciestunepipe.file import bcistructure as et

reload(et)
sess_par = {'bird': 's_b1253_21',
           'sess': 'dummy',
           'ephys_software': 'alsa',
           'bouts_detector': 'bouts_ceciestunepipe',
           'bouts_file': 'bout_checked.pickle'
}


bird_sessions = et.list_sessions(sess_par['bird'], ephys_software='alsa')

In [16]:
### list bird sessions with curated bouts

In [25]:
def list_bird_bout_files(sess_par: dict):
    # list all the sessions for the bird
    bird_sessions = et.list_sessions(sess_par['bird'], ephys_software=sess_par['ephys_software'])
    bird_sessions.sort()

    
    # for each session, look whether there is a bout_checked_file
    all_bout_path = []
    for sess in bird_sessions:
        exp_struct = et.get_exp_struct(sess_par['bird'], sess, ephys_software=sess_par['ephys_software'])
        
        bouts_folder = os.path.join(exp_struct['folders']['derived'], sess_par['bouts_detector'])
        bouts_file_path = os.path.join(bouts_folder, sess_par['bouts_file'])
        #logger.info(bouts_file_path)
        if os.path.isfile(bouts_file_path):
            all_bout_path.append(bouts_file_path)
    return all_bout_path
            
bird_bout_file_list = list_bird_bout_files(sess_par)
        

In [26]:
bird_bout_file_list

['/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-06/alsa/bouts_ceciestunepipe/bout_checked.pickle',
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-12/alsa/bouts_ceciestunepipe/bout_checked.pickle',
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-13/alsa/bouts_ceciestunepipe/bout_checked.pickle',
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/alsa/bouts_ceciestunepipe/bout_checked.pickle',
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-15/alsa/bouts_ceciestunepipe/bout_checked.pickle',
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-16/alsa/bouts_ceciestunepipe/bout_checked.pickle',
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-17/alsa/bouts_ceciestunepipe/bout_checked.pickle',
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-18/alsa/bouts_ceciestunepipe/bout_checked.pickle',
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-29/alsa/bouts_ceciestunepipe/bout_checked.pickle',
 '/mnt/sphere/speech_bci/der

In [38]:
from tqdm.notebook import trange, tqdm

In [52]:
def load_all_bout_pandas(sess_par: dict):
    # list all the bout files in the session
    path_list = list_bird_bout_files(sess_par)
    # load them all and concatenate into one single pandas dataframe
    pd_list = []
    logger.info('loading all pandas {} for bird {}'.format(sess_par['bouts_file'], sess_par['bird']))
    for i_p, one_path in enumerate(tqdm(path_list)):
        one_pd = pd.read_pickle(one_path)
        pd_list.append(one_pd)
    
    return pd.concat(pd_list).reset_index(drop=True)

bird_bout_pd = load_all_bout_pandas(sess_par)
        

2021-07-06 00:07:15,784 root         INFO     loading all pandas bout_checked.pickle for bird s_b1253_21


  0%|          | 0/11 [00:00<?, ?it/s]

In [56]:
bird_bout_pd

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram,confusing
0,224595.0,251655.0,10780560.0,12079440.0,"[2.194746139749094, 3.850394476655695, 2.16753...",3.837231,104.672704,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,27060.0,"[[224595, 224700], [226320, 226465], [227510, ...",73.0,"[1, 10, 18, 32, 41, 348, 356, 364, 373, 386, 3...",240.0,112.750000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2, -2, -1, -1,...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
1,924555.0,945395.0,44378640.0,45378960.0,"[2.6427765346156753, 3.1863976015258806, 4.622...",3.837231,115.114813,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,20840.0,"[[924555, 924635], [925235, 925290], [925315, ...",82.0,"[7, 14, 31, 76, 145, 155, 162, 173, 201, 239, ...",233.0,89.442060,"[-5, -4, -4, -3, -4, -4, -4, -6, -4, -2, -3, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
2,1041975.0,1071595.0,50014800.0,51436560.0,"[26.3627577571065, 27.576856238944778, 24.7911...",3.837231,164.391799,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,29620.0,"[[1041975, 1042015], [1042045, 1042075], [1042...",93.0,"[1, 16, 57, 126, 162, 170, 187, 232, 243, 293,...",324.0,91.419753,"[3, 3, 1, 1, 2, 1, 1, 1, 3, 4, 3, 3, 2, 2, 1, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
3,1280975.0,1314870.0,61486800.0,63113760.0,"[2.952162417231377, 3.6333684720191672, 2.9426...",3.837231,169.376257,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,33895.0,"[[1280975, 1281005], [1282260, 1282355], [1282...",142.0,"[1, 20, 117, 139, 185, 263, 273, 337, 400, 409...",451.0,75.155211,"[22, 26, 30, 31, 26, 25, 18, 13, 8, -1, -11, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
4,1384650.0,1412995.0,66463200.0,67823760.0,"[2.3698052125938878, 2.568265352682433, 1.4532...",3.837231,135.384378,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,28345.0,"[[1384650, 1384685], [1385635, 1385665], [1385...",79.0,"[1, 21, 28, 71, 80, 94, 113, 165, 172, 201, 21...",237.0,119.599156,"[-12, -14, -13, -11, -6, 3, 11, 19, 24, 23, 20...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,55050.0,93975.0,2642400.0,4510800.0,"[4.064574659563392, 4.78674915045126, 4.877457...",7.314456,169.716511,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,38925.0,"[[55050, 55165], [55955, 56005], [56015, 57085...",151.0,"[2, 16, 188, 208, 216, 224, 240, 254, 264, 273...",444.0,87.668919,"[-5, -5, -5, -6, -4, -5, -5, -5, -4, -4, -4, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
830,199460.0,265380.0,9574080.0,12738240.0,"[4.211330667954928, 6.520441351789098, 6.57156...",7.314456,174.896675,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,65920.0,"[[199460, 199565], [201455, 202550], [204090, ...",209.0,"[2, 13, 410, 419, 432, 442, 452, 461, 468, 481...",622.0,105.980707,"[-4, -5, -5, -3, -4, -5, -5, -6, -6, -6, -5, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
831,721960.0,768230.0,34654080.0,36875040.0,"[3.6354642773772547, 3.3828416268779016, 2.386...",7.314456,161.406485,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,46270.0,"[[721960, 722000], [722010, 722050], [723695, ...",139.0,"[3, 15, 352, 426, 598, 723, 981, 1077, 1180, 1...",321.0,144.143302,"[3, 3, 3, 3, 3, 3, 2, 1, 1, 2, 2, 2, 3, 4, 3, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
832,1230820.0,1292120.0,59079360.0,62021760.0,"[3.4879651118976107, 22.627850898522702, 43.34...",7.314456,171.312220,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,61300.0,"[[1230820, 1230850], [1232815, 1232855], [1233...",199.0,"[2, 155, 406, 446, 486, 600, 730, 979, 1227, 1...",565.0,108.495575,"[-13, -15, -12, -8, -1, 4, 6, 13, 18, 19,

##### remove all the 'confusing' or noise ('!bout_check')

In [53]:
only_bout_pd = bird_bout_pd.drop(bird_bout_pd[(bird_bout_pd['confusing']==True) | (bird_bout_pd['bout_check']==False)].index)

In [54]:
only_bout_pd

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram,confusing
0,224595.0,251655.0,10780560.0,12079440.0,"[2.194746139749094, 3.850394476655695, 2.16753...",3.837231,104.672704,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,27060.0,"[[224595, 224700], [226320, 226465], [227510, ...",73.0,"[1, 10, 18, 32, 41, 348, 356, 364, 373, 386, 3...",240.0,112.750000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2, -2, -1, -1,...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
1,924555.0,945395.0,44378640.0,45378960.0,"[2.6427765346156753, 3.1863976015258806, 4.622...",3.837231,115.114813,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,20840.0,"[[924555, 924635], [925235, 925290], [925315, ...",82.0,"[7, 14, 31, 76, 145, 155, 162, 173, 201, 239, ...",233.0,89.442060,"[-5, -4, -4, -3, -4, -4, -4, -6, -4, -2, -3, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
2,1041975.0,1071595.0,50014800.0,51436560.0,"[26.3627577571065, 27.576856238944778, 24.7911...",3.837231,164.391799,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,29620.0,"[[1041975, 1042015], [1042045, 1042075], [1042...",93.0,"[1, 16, 57, 126, 162, 170, 187, 232, 243, 293,...",324.0,91.419753,"[3, 3, 1, 1, 2, 1, 1, 1, 3, 4, 3, 3, 2, 2, 1, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
3,1280975.0,1314870.0,61486800.0,63113760.0,"[2.952162417231377, 3.6333684720191672, 2.9426...",3.837231,169.376257,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,33895.0,"[[1280975, 1281005], [1282260, 1282355], [1282...",142.0,"[1, 20, 117, 139, 185, 263, 273, 337, 400, 409...",451.0,75.155211,"[22, 26, 30, 31, 26, 25, 18, 13, 8, -1, -11, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
4,1384650.0,1412995.0,66463200.0,67823760.0,"[2.3698052125938878, 2.568265352682433, 1.4532...",3.837231,135.384378,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,28345.0,"[[1384650, 1384685], [1385635, 1385665], [1385...",79.0,"[1, 21, 28, 71, 80, 94, 113, 165, 172, 201, 21...",237.0,119.599156,"[-12, -14, -13, -11, -6, 3, 11, 19, 24, 23, 20...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828,1741400.0,1774055.0,83587200.0,85154640.0,"[2.0308561055381595, 3.4650639989272727, 2.752...",5.179853,110.958443,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,32655.0,"[[1741400, 1741505], [1742160, 1742190], [1742...",118.0,"[7, 14, 21, 153, 192, 200, 212, 223, 236, 245,...",374.0,87.312834,"[7, 7, 6, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 5, 6, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
829,55050.0,93975.0,2642400.0,4510800.0,"[4.064574659563392, 4.78674915045126, 4.877457...",7.314456,169.716511,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,38925.0,"[[55050, 55165], [55955, 56005], [56015, 57085...",151.0,"[2, 16, 188, 208, 216, 224, 240, 254, 264, 273...",444.0,87.668919,"[-5, -5, -5, -6, -4, -5, -5, -5, -4, -4, -4, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
830,199460.0,265380.0,9574080.0,12738240.0,"[4.211330667954928, 6.520441351789098, 6.57156...",7.314456,174.896675,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,65920.0,"[[199460, 199565], [201455, 202550], [204090, ...",209.0,"[2, 13, 410, 419, 432, 442, 452, 461, 468, 481...",622.0,105.980707,"[-4, -5, -5, -3, -4, -5, -5, -6, -6, -6, -5, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
831,721960.0,768230.0,34654080.0,36875040.0,"[3.6354642773772547, 3.3828416268779016, 2.386...",7.314456,161.406485,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,46270.0,"[[721960, 722000], [722010, 722050], [723695, ...",139.0,"[3, 15, 352, 426, 598, 723, 981, 1077, 1180, 1...",321.0,144.143302,"[3, 3, 3, 3, 3, 3, 2, 1, 1, 2, 2, 2, 3, 4,

### save it somewhere

In [57]:
bout_folder = '/mnt/sphere/speech_bci/analysis/bout/s_b1253_21'
bout_file = 'check_bouts.pickle'

only_bout_pd.to_pickle(os.path.join(bout_folder, bout_file))

In [58]:
os.path.join(bout_folder, bout_file)

'/mnt/sphere/speech_bci/analysis/bout/s_b1253_21/check_bouts.pickle'

### save each bout to a wav file

In [59]:
one_bout = only_bout_pd.iloc[0]
one_bout

start_ms                                                   224595.0
end_ms                                                     251655.0
start_sample                                             10780560.0
end_sample                                               12079440.0
p_step            [2.194746139749094, 3.850394476655695, 2.16753...
rms_p                                                      3.837231
peak_p                                                   104.672704
bout_check                                                     True
file              /mnt/sphere/speech_bci/raw_data/s_b1253_21/202...
len_ms                                                      27060.0
syl_in            [[224595, 224700], [226320, 226465], [227510, ...
n_syl                                                          73.0
peaks_p           [1, 10, 18, 32, 41, 348, 356, 364, 373, 386, 3...
n_peaks                                                       240.0
l_p_ratio                                       

In [86]:
np.where(only_bout_pd['valid'] == False)

(array([], dtype=int64),)

In [67]:
one_bout.index

Index(['start_ms', 'end_ms', 'start_sample', 'end_sample', 'p_step', 'rms_p',
       'peak_p', 'bout_check', 'file', 'len_ms', 'syl_in', 'n_syl', 'peaks_p',
       'n_peaks', 'l_p_ratio', 'waveform', 'valid_waveform', 'valid',
       'spectrogram', 'confusing'],
      dtype='object')

In [71]:
fpath, fname = os.path.split(one_bout['file'])

fpath.split(os.path.sep)

['',
 'mnt',
 'sphere',
 'speech_bci',
 'raw_data',
 's_b1253_21',
 '2021-06-06',
 'alsa']

In [73]:
wavfile.write?

In [91]:
bout_wav_folder = os.path.join(bout_folder, 'bout_wav')

def bout_to_wav(bout_s: pd.Series, s_f=48000, border_ms=1000):
    # trim the edges
    border_samples = int(s_f * border_ms * 0.001)
    # get the wav for the series (trim the edges)
    #x = bout_s['waveform'][border_samples: -border_samples]
    x = bout_s['waveform']
    #define wav file
    path_parts, path_name = os.path.split(bout_s['file'])
    [bird, sess] = path_parts.split(os.path.sep)[5:7]
    file = path_name.split('.wav')[0]
    
    out_fname = '{}-{}-{}-{}.wav'.format(bird, sess, file, int(bout_s['start_sample']))
    
    #save as wav
    out_wav_path = os.path.join(bout_folder, 'bout_wav', out_fname)
    os.makedirs(os.path.split(out_wav_path)[0], exist_ok=True)
    #logger.info('saving to ' + out_wav_path)
    wavfile.write(out_wav_path, s_f, x)
    return out_wav_path
    
## these come from the params pickle file
s_f = 48000
border_ms = 900

bout_to_wav(one_bout, s_f=s_f, border_ms=border_ms)


'/mnt/sphere/speech_bci/analysis/bout/s_b1253_21/bout_wav/s_b1253_21-2021-06-06-07-00-01-01-10780560.wav'

In [92]:
## save all
only_bout_pd.apply(lambda s: bout_to_wav(s, s_f=s_f, border_ms=border_ms), axis=1)

TypeError: expected str, bytes or os.PathLike object, not float